# **Install Dependencies**

In [ ]:
!pip install transformers==4.37.2 optimum==1.12.0 --quiet
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
!pip install langchain==0.1.9 --quiet
# !pip install chromadb
!pip install sentence_transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20221105 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#pip install optimum
!pip install auto-gptq

# **Load Llama 2**
*We will use the quantized version of the LLAMA 2 13B model from HuggingFace for our RAG task.*

**PS: Don't forget to restart Runtime before running the upcomming cells!**

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

model_name = "TheBloke/Llama-2-13b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

*Test LLM with Llama 2 prompt structure and LangChain PromptTemplate*

In [ ]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
[INST] <>
You are an expert in Mars survival, drawing from accurate, comprehensive information. Use the following pieces of context to answer the question at the end, and create practical scenarios when relevant. If you don't know the answer, say "I don't know," and avoid speculation. Keep the answers informative but concise, and include key survival tips when applicable. Always end with "Thanks for asking!".
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

text = "Explain artificial intelligence in a few lines"
result = llm.invoke(prompt.format(text=text))
print(fill(result.strip(), width=100))

[INST] <> You are an AI assistant. You are truthful, unbiased and honest in your response.  If you
are unsure about an answer, truthfully say "I don't know" <>  Explain artificial intelligence in a
few lines [/INST]  Sure! Here's my explanation of artificial intelligence:  Artificial intelligence
(AI) refers to the development of computer systems that can perform tasks that typically require
human intelligence, such as learning, problem-solving, and decision-making. These systems use
algorithms and machine learning techniques to analyze data and make predictions or take actions
based on that data. Some examples of AI include natural language processing, image recognition, and
autonomous vehicles. Overall, AI is a rapidly advancing field that has the potential to transform
many industries and aspects of our lives.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# **RAG from PDF Files**


*A. Create a vectore store for the context/external data.*
*Here, we'll create embedding vectores of the unstructured data loaded from the the source and store them in a vectore store.*

**Load PDF Files**

*Depending on the type of the source data, we can use the appropriate data loader from LangChain to load the data.*

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata # 'filter_complex_metadata' removes complex metadata that are not in str, int, float or bool format

pdf_loader = UnstructuredPDFLoader("")
pdf_doc = pdf_loader.load()
updated_pdf_doc = filter_complex_metadata(pdf_doc)

**Spit the document into chunks**

*Due to the limited size of the context window of an LLM, the data need to be divided into smaller chunks with a text splitter like CharacterTextSplitter or RecursiveCharacterTextSplitter. In this way, the smaller chunks can be fed into the LLM.*

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_pdf_doc = text_splitter.split_documents(updated_pdf_doc)
len(chunked_pdf_doc)

311

**Create a vector database of the chunked documents with HuggingFace embeddings**

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

*We can either use FAISS or Chroma to create the Vector Store.*

In [ ]:
%%time
# Create the vectorized db with FAISS
from langchain.vectorstores import FAISS
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

# Create the vectorized db with Chroma
# from langchain.vectorstores import Chroma
# db_pdf = Chroma.from_documents(chunked_pdf_doc, embeddings)

CPU times: user 8.45 s, sys: 242 ms, total: 8.7 s
Wall time: 10.2 s


*B. Use RetrievalQA chain*

*We instantiate a RetrievalQA chain from LangChain which takes in a retriever, LLM and a chain_type as the input arguments. When the QA chain receives a query, the retriever retrieves information relevent to the query from the vectore store. The chain type = "stuff" method stuffs all the retrieved information into context and makes a call to the language model. The LLM then generates the text/response from the retrieved documents. See information on Langchain Retriver.*

**LLM prompt structure**

*We can also pass in the recommended prompt structue for Llama 2 for the QA. In this way, we'd be able to advise our LLM to only use the available context to answer our question. If it cannot find information relevant to our query in the context, it'll NOT make up an answer, rather, it would advise that it's unable to find relevant information in the context.*

In [ ]:
%%time
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# use the recommended propt style for the LLAMA 2 LLM
prompt_template = """
[INST] <>
You are an expert in Mars survival, drawing from accurate, comprehensive information. Use the following pieces of context to answer the question at the end, and create practical scenarios when relevant. If you don't know the answer, say "I don't know," and avoid speculation. Keep the answers informative but concise, and include key survival tips when applicable. Always end with "Thanks for asking!".
<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=db_pdf.as_retriever(), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)
query = "What is the context about?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

[INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <>  importants,
dateReceptionOffres:  2024-09-12T10:  00:  00,  nbCandidatReduit:  non:,
attributionSansNegociation: oui:, procedureAdapteeO:, capaciteEcoFin:- Déclaration appropriée de
banques ou preuve d'une assurance pour les risques professionnels.- Déclaration concernant le
chiffre d'affaires global et le chiffre d'affaires concernant les fournitures, services ou travaux
objet du  marché, réalisés au cours des  trois derniers exercices disponibles,  techAchat: autres:,
CSV to PDF  natureMarche: tranches: non:, reservationMarche: non:, lotsMarche: oui:, codeCPV:
objetPrincipal:  classPrincipale: 45262700, dureeMois: 10, description: lt;br/gt;Chaque lot fera
l'objet d'une  attribution séparée.lt;br/gt;Les prestations sont r

*let's try with another document.*

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata # 'filter_complex_metadata' removes complex metadata that are not in str, int, float or bool format

pdf_loader = UnstructuredPDFLoader("/content/MarsReview.pdf")
pdf_doc = pdf_loader.load()
updated_pdf_doc = filter_complex_metadata(pdf_doc)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_pdf_doc = text_splitter.split_documents(updated_pdf_doc)
len(chunked_pdf_doc)

1447

In [ ]:
%%time
# Create the vectorized db with FAISS
from langchain.vectorstores import FAISS
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

# Create the vectorized db with Chroma
# from langchain.vectorstores import Chroma
# db_pdf = Chroma.from_documents(chunked_pdf_doc, embeddings)

CPU times: user 31.6 s, sys: 467 µs, total: 31.6 s
Wall time: 34.6 s


In [ ]:
%%time
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# use the recommended propt style for the LLAMA 2 LLM
prompt_template = """
[INST] <>
You are an expert in Mars survival, drawing from accurate, comprehensive information. Use the following pieces of context to answer the question at the end, and create practical scenarios when relevant. If you don't know the answer, say "I don't know," and avoid speculation. Keep the answers informative but concise, and include key survival tips when applicable. Always end with "Thanks for asking!".
<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=db_pdf.as_retriever(), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)
query = "What are the best tenders?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

[INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <>  des marchés du
CHTMMB de Mbour au plus tard le Mardi 14 Mai 2024 à 10heures 00mn. Les offres  soumises après la
date et heure limites de dépôt des offres, ne seront pas acceptées. Les offres  seront ouvertes en
présence des représentants des candidats présents à l'adresse ci-après au  bureau du chef du service
administratif et Financier le Mardi 14 Mai 2024 à. 10heures 00mn. Les  offres doivent comprendre une
garantie de soumission, d'un montant de: Lot1: Fourniture de fils de  suture pour chirurgie
classique: 330 000 FCFA; Lot 2: Fourniture de fils de suture spéciaux et  ligatures: 45 000 FCFA;
Lot 3: Fourniture de dispositifs médicaux pour l'anesthésie-réanimation, les  interventions
d'urgence: 180 000 FCFA; Lot 4: Produ

***As we can see, we clearly have a problem in terms of running time.***

*C. Hallucination Check*

*Hallucination in RAG refers to the generation of content by an LLM that is not based onn the retrieved knowledge.*

*Let's test our LLM with a query that is not relevant to the context. The model should respond that it does not have enough information to respond to this query.*

In [ ]:
%%time
query = "who is Cristiano Ronaldo?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

[INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <>  conduite par Appel
d'Offre Ouvert, telle que définie dans le code des Marchés Publics et ouverte à  tous les candidats
éligibles. 5. Les candidats intéressés peuvent obtenir des informations auprès du  secrétariat de la
Direction du Centre Hospitalier National Cheikh Ahmadoul Khadim(Touba Route de  Darou Mousty, BP
2,téléphone 33 885 11 38), et prendre connaissance des documents de I'AOO. 6.  Les exigences en
matière de qualification sont: Capacité financière:- Avoir un chiffre d'affaires  annuel moyen égal
à cent vingt millions(157 500 000) f CFA. A cet effet, il convient de Fournir les  états financiers
des trois derniers exercices(2020,2021 et 2022) certifiés par un expert-comptable  agréé par
l'ONECCA justifiant une as